# Test notebook

In [1]:
library(ArchR)
library(org.Ss.eg.db)
library(SuscrofaTxdb.11.108.july)
library(Repitools)


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
                ==###########>      /                   ||
                 \\##==......\    /                     ||
            ______ =       =|__ /__                     ||      \\\
        ,--' ,----`-,__ ___/'  --,-`-===================##========>
       \               '        ##_______ _____ ,--,__,=##,__   ///
        ,    __==    ___,-,__,--'#'  ==='      `-'    | ##,-/
        -,____,---'       \\####\\________________,--\\_##,/
           ___      .______        ______  __    __  .______      
          /   \     |   _ 

In [2]:
options(repr.plot.width = 18, repr.plot.height = 17, repr.plot.pointsize = 24)

In [3]:
addArchRThreads(2)

Setting default number of Parallel threads to 2.



In [4]:
load(file = "/home/adufour/work/rds_storage/omics/archr_all_v7_embryo.RData")

In [5]:
peak_grange <- archrproj_sub@peakSet

In [6]:
peak_grange$peakName <- peak_grange %>% {paste0(seqnames(.), "_", start(.), "_", end(.))}

In [7]:
names(peak_grange) <- NULL

In [8]:
peak_DF <- annoGR2DF(peak_grange)

In [9]:
peak_DF

chr,start,end,width,score,replicateScoreQuantile,groupScoreQuantile,Reproducibility,GroupReplicate,distToGeneStart,nearestGene,peakType,distToTSS,nearestTSS,GC,idx,N,peakName
<fct>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<dbl>,<int>,<dbl>,<chr>
1,5561,6061,501,13.29980,0.664,0.548,4,C4._.J11-2,4921,ENSSSCG00000048769,Distal,4921,ENSSSCT00000078085,0.4291,1,0,1_5561_6061
1,19545,20045,501,11.57550,0.553,0.400,3,C4._.J11-1,2356,ENSSSCG00000037372,Intronic,2356,ENSSSCT00000046027,0.4371,2,0,1_19545_20045
1,21957,22457,501,51.98670,0.993,0.983,5,C4._.J9-1,54,ENSSSCG00000037372,Promoter,54,ENSSSCT00000046027,0.5649,3,0,1_21957_22457
1,23527,24027,501,39.47710,0.907,0.856,5,C4._.J11-2,100,PSMB1,Promoter,6,ENSSSCT00000058189,0.5768,4,0,1_23527_24027
1,98294,98794,501,17.63700,0.610,0.157,2,C3._.Rep1,15466,ENSSSCG00000049216,Intronic,15466,ENSSSCT00000068313,0.6228,5,0,1_98294_98794
1,139826,140326,501,6.23646,0.541,0.246,2,C1._.J7,26064,ENSSSCG00000049216,Intronic,26064,ENSSSCT00000068313,0.6148,6,0,1_139826_140326
1,140973,141473,501,5.10796,0.454,0.221,2,C1._.J7,27211,ENSSSCG00000049216,Intronic,27211,ENSSSCT00000068313,0.5669,7,0,1_140973_141473
1,165827,166327,501,11.44310,0.737,0.476,2,C1._.J11-4,20691,FAM120B,Intronic,9656,ENSSSCT00000093585,0.5828,8,0,1_165827_166327
1,185846,186346,501,10.49230,0.522,0.354,2,C4._.J11-1,672,FAM120B,Intronic,619,ENSSSCT00000055371,0.5050,9,0,1_185846_186346


In [10]:
markersPeaks <- getMarkerFeatures(
    ArchRProj = archrproj_sub, 
    useMatrix = "PeakMatrix", 
    groupBy = "Clusters",
    bias = c("TSSEnrichment", "log10(nFrags)"),
    testMethod = "wilcoxon"
)

ArchR logging to : ArchRLogs/ArchR-getMarkerFeatures-3e1f5e496b0dcc-Date-2024-03-15_Time-16-13-08.log
If there is an issue, please report to github with logFile!

MatrixClass = Sparse.Integer.Matrix

2024-03-15 16:13:10 : Matching Known Biases, 0.025 mins elapsed.

Warning message:
“le package ‘nabor’ a été compilé avec la version R 4.2.3”
###########
2024-03-15 16:39:20 : Completed Pairwise Tests, 26.189 mins elapsed.
###########

ArchR logging successful to : ArchRLogs/ArchR-getMarkerFeatures-3e1f5e496b0dcc-Date-2024-03-15_Time-16-13-08.log



# export DAG whole

In [11]:
scriptPath <- "/home/adufour/work/scScalpChromatin"
source(paste0(scriptPath, "/misc_helpers.R"))
source(paste0(scriptPath, "/matrix_helpers.R"))
source(paste0(scriptPath, "/plotting_config.R"))
source(paste0(scriptPath, "/archr_helpers.R"))

Warning message:
“le package ‘dplyr’ a été compilé avec la version R 4.2.3”
Warning message:
“le package ‘tidyr’ a été compilé avec la version R 4.2.3”
Warning message:
“le package ‘RcppAlgos’ a été compilé avec la version R 4.2.3”
Warning message:
“le package ‘ggrastr’ a été compilé avec la version R 4.2.3”


In [12]:
corrCutoff <- 0.5
p2gGR <- getP2G_GR(archrproj_sub, corrCutoff=corrCutoff)

In [13]:
p2geneDF <- Repitools::annoGR2DF(p2gGR)

In [14]:
markerPeaksList <- getMarkers(markersPeaks, cutOff = "FDR <= 0.1 & abs(Log2FC) >= 1")

In [24]:
markerPeaksList

List of length 5
names(5): C1 C2 C3 C4 C6

In [25]:
markerPeaksList$C1$peakName <- paste0(markerPeaksList$C1$seqnames, "_", markerPeaksList$C1$start, "_", markerPeaksList$C1$end)

In [26]:
markerPeaksList$C2$peakName <- paste0(markerPeaksList$C2$seqnames, "_", markerPeaksList$C2$start, "_", markerPeaksList$C2$end)

In [27]:
markerPeaksList$C3$peakName <- paste0(markerPeaksList$C3$seqnames, "_", markerPeaksList$C3$start, "_", markerPeaksList$C3$end)

In [28]:
markerPeaksList$C4$peakName <- paste0(markerPeaksList$C4$seqnames, "_", markerPeaksList$C4$start, "_", markerPeaksList$C4$end)

In [18]:
peak_grange <- archrproj_sub@peakSet

In [19]:
peak_grange$peakName <- peak_grange %>% {paste0(seqnames(.), "_", start(.), "_", end(.))}

In [20]:
names(peak_grange) <- NULL

In [21]:
peak_DF <- Repitools::annoGR2DF(peak_grange)

In [29]:
merge_C1 <- merge(markerPeaksList$C1,p2geneDF,by="peakName", all.x=TRUE)
merge_C2 <- merge(markerPeaksList$C2,p2geneDF,by="peakName", all.x=TRUE)
merge_C3 <- merge(markerPeaksList$C3,p2geneDF,by="peakName", all.x=TRUE)
merge_C4 <- merge(markerPeaksList$C4,p2geneDF,by="peakName", all.x=TRUE)

In [30]:
write.csv(merge(merge_C1,peak_DF,by="peakName"), "/home/adufour/work/table/omics_embryo_dag_c1.csv", row.names=TRUE)
write.csv(merge(merge_C2,peak_DF,by="peakName"), "/home/adufour/work/table/omics_embryo_dag_c2.csv", row.names=TRUE)
write.csv(merge(merge_C3,peak_DF,by="peakName"), "/home/adufour/work/table/omics_embryo_dag_c3.csv", row.names=TRUE)
write.csv(merge(merge_C4,peak_DF,by="peakName"), "/home/adufour/work/table/omics_embryo_dag_c4.csv", row.names=TRUE)

In [10]:
write.csv(peak_DF, "/home/adufour/work/table/peak_DF_stemcells.csv", row.names=FALSE)